<a href="https://colab.research.google.com/github/surabhiwaingankar/cart_pole_v0/blob/main/cart_pole_rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing Dependancies**


In [34]:
!pip install stable-baselines3[extra]

In [35]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import pygame


**Load Environment**

In [36]:
enviroment_name = 'CartPole-v0'
env = gym.make(enviroment_name)

/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


**Testing environment**

In [37]:
pygame.init()
screen = pygame.display.set_mode((600, 400))
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        screen.fill((255, 255, 255))
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info, _ = env.step(action)
        score+=reward
        pygame.display.flip()
    print('Episode:{} Score:{}'.format(episode, score))
env.close()
pygame.quit()

/usr/local/lib/python3.10/dist-packages/gymnasium/envs/classic_control/cartpole.py:215: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v0", render_mode="rgb_array")
  gym.logger.warn(


Episode:1 Score:12.0
Episode:2 Score:26.0
Episode:3 Score:36.0
Episode:4 Score:17.0
Episode:5 Score:17.0


env.reset() resets the env to original state of values of agent,env, action and outcome

In [38]:
env.reset()

(array([-0.0066784 , -0.03439371, -0.02872949,  0.0310617 ], dtype=float32),
 {})

env.action_space returns what actioncan be taken- here 2 actions, i.e left(0)/right(1) so Discrete(2)
env.action_space.sample() which can be 0,1 (2 discrete values)

In [39]:
env.action_space

Discrete(2)

In [40]:
env.action_space.sample()

1

env.observation_space returns observations for env

In [41]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

cart_position, cart_velocity, pole-angle and pole-angular velocity are 4 observations here for this usecase(from openai gym)

env.step returns us state, reward, done and info- reward is 0/1, done is True/false signifying if episode is completed or not, state is the state of 4 variables

env has action space and observation space

**Training Environment**

In [42]:
log_path = os.path.join('Training', 'Logs')


In [43]:
env = gym.make(enviroment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


env = DummyVecEnv([lambda: env]) this wraps up env in an vector environment to test
Mlp - multilayer percepion policy followed
tensorboard_log contains logs
verbose = 1 as we want to log



In [44]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/PPO_3
-----------------------------
| time/              |      |
|    fps             | 1120 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 752         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009125682 |
|    clip_fraction        | 0.119       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.685      |
|    explained_variance   | -0.00306    |
|    learning_rate        | 0.0003      |
|    loss                 | 8.24        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0201     |
|    value_loss           | 54.1        |
-----------------------------------------
---

In [45]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [46]:
model.save(PPO_path)

In [47]:
del model

In [48]:
model = PPO.load("Training/Saved Models/PPO_model", env=env)

**Evaluate**

In [49]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

**Testing the model**

In [50]:
pygame.init()
screen = pygame.display.set_mode((600, 400))
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        screen.fill((255, 255, 255))
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score+=reward
        pygame.display.flip()
    print('Episode:{} Score:{}'.format(episode, score))
env.close()
pygame.quit()

Episode:1 Score:[200.]
Episode:2 Score:[81.]
Episode:3 Score:[200.]
Episode:4 Score:[88.]
Episode:5 Score:[200.]


In [51]:
obs = env.reset()

In [52]:
model.predict(obs)

(array([0]), None)

**Adding callback to training stage**

In [53]:

from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
save_path = os.path.join('Training', 'Saved Models')
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path=save_path,
                             verbose=1)

In [54]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [55]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/PPO_4
-----------------------------
| time/              |      |
|    fps             | 933  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 743        |
|    iterations           | 2          |
|    time_elapsed         | 5          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00941643 |
|    clip_fraction        | 0.121      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.686     |
|    explained_variance   | -0.00178   |
|    learning_rate        | 0.0003     |
|    loss                 | 6.17       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0191    |
|    value_loss           | 50.6       |
----------------------------------------
---------------------

**Changing Policies**

We can specify a new neural network architechture- by changing the number of units and layers to aur neural network

In [56]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

4 layers with 128 units in each layer

In [57]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

Using cpu device


In [58]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 963  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 620        |
|    iterations           | 2          |
|    time_elapsed         | 6          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01574736 |
|    clip_fraction        | 0.237      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.681     |
|    explained_variance   | -0.00581   |
|    learning_rate        | 0.0003     |
|    loss                 | 2.41       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0241    |
|    value_loss           | 16.7       |
----------------------------------------
-----------------------------------------
| time/   

**Using an alternate algorithm**

In [59]:
from stable_baselines3 import DQN

instead of PPO we take DQN

In [60]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [61]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/DQN_2
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.943    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 4815     |
|    time_elapsed     | 0        |
|    total_timesteps  | 119      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.884    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 5877     |
|    time_elapsed     | 0        |
|    total_timesteps  | 244      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.854    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 5814     |
|    time_elapsed     | 0        |
|    total_timesteps  | 308      |
----------------------------------
------------------------

In [62]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [63]:
model.save(dqn_path)

In [64]:
model = DQN.load(dqn_path, env=env)

In [65]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(9.8, 1.16619037896906)

In [66]:
env.close()